In [ ]:
# ติดตั้งไลบรารีที่จำเป็นสำหรับ RAG เช่น:
# transformers, langchain: สำหรับใช้โมเดลและสร้างโซ่การทำงาน
# chromadb: จัดเก็บและค้นหา embedding
# sentence_transformers, InstructorEmbedding: แปลงข้อความเป็นเวกเตอร์
# pypdf, pdf2image: อ่านไฟล์ PDF
# xformers, bitsandbytes, accelerate: ช่วยให้โมเดลทำงานเร็วขึ้น (โดยเฉพาะกับ GPTQ)
# pydantic: ใช้สำหรับจัดการข้อมูลแบบ type-safe

In [ ]:
import torch
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from pdf2image import convert_from_path
from transformers import AutoTokenizer, TextStreamer, pipeline, AutoModelForCausalLM

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"

## Data

In [ ]:
# แปลงไฟล์ PDF ให้กลายเป็นรูปภาพทีละหน้า ด้วยไลบรารี pdf2image
from pdf2image import convert_from_path

meta_images = convert_from_path(
    "pdfs/tesla-earnings-report.pdf",
    poppler_path = r"C:\tools\poppler-24.08.0\Library\bin", 
    dpi=88
)
meta_images[0]

In [ ]:
# "pdfs/tesla-earnings-report.pdf"
# 👉 ไฟล์ PDF ที่จะนำมาแปลงเป็นรูปภาพ
# poppler_path
# 👉 เส้นทางไปยังไฟล์ Poppler บน Windows (จำเป็นสำหรับ pdf2image)
# dpi=88
# 👉 ความละเอียดของภาพ (Dots Per Inch)
# ค่ายิ่งสูง ภาพยิ่งชัด แต่ใช้หน่วยความจำมากขึ้น

In [ ]:
nvidia_images = convert_from_path(
    "pdfs/nvidia-earnings-report.pdf",
    poppler_path = r"C:\tools\poppler-24.08.0\Library\bin", 
    dpi=88
)
nvidia_images[0]

In [ ]:
tesla_images = convert_from_path(
    "pdfs/tesla-earnings-report.pdf",
    poppler_path = r"C:\tools\poppler-24.08.0\Library\bin", 
    dpi=88
)
tesla_images[0]

In [ ]:
# remove vector db every time when running the workshop.
import shutil
shutil.rmtree("db", ignore_errors=True)

In [ ]:
# load all pdf file in a folder pdfs.
loader = PyPDFDirectoryLoader("pdfs")
docs = loader.load()
len(docs)

In [ ]:
# สร้าง Embedding Model ด้วย Hugging Face ผ่าน LangChain เพื่อนำข้อความไปแปลงเป็นเวกเตอร์ (Vector) สำหรับใช้ใน RAG pipeline เช่น การค้นหาเอกสารที่เกี่ยวข้อง (retrieval)
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(
    model_name="intfloat/multilingual-e5-large", 
    model_kwargs={"device": DEVICE}
)

In [ ]:
# model_name="intfloat/multilingual-e5-large"
# 👉 เป็นชื่อโมเดลฝั่ง Embedding ที่จะโหลดจาก Hugging Face
# โมเดลนี้รองรับหลายภาษา (multilingual) และมีประสิทธิภาพดีในการแปลงข้อความเป็นเวกเตอร์

# model_kwargs={"device": DEVICE}  ( Model Keyword Arguments )
# 👉 กำหนดว่าให้รันบนอุปกรณ์ใด เช่น "cuda" หรือ "cpu"
# โดย DEVICE อาจจะตั้งไว้ก่อนหน้านี้ เช่น:

In [ ]:
# สร้างตัวแยกข้อความ (Text Splitter) ที่จะใช้แบ่งเอกสารออกเป็นส่วนย่อย ๆ (chunk) เพื่อให้โมเดลประมวลผลได้ง่ายขึ้น
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=64)
texts = text_splitter.split_documents(docs)
len(texts)

In [ ]:
# กำหนด text_splitter
# chunk_size=1024 → แต่ละส่วนมีความยาวสูงสุด 1024 charectors.
# chunk_overlap=64 → แต่ละส่วนจะมีเนื้อหาทับซ้อนกับส่วนก่อนหน้า 64 ตัวอักษร เพื่อให้การเข้าใจบริบทดีขึ้น

# texts = text_splitter.split_documents(docs) => นำ docs (ไฟล์เอกสาร PDF ที่แปลงเป็นข้อความไว้ก่อนหน้า) มาแบ่งออกเป็นหลาย ๆ ส่วน โดยใช้ text_splitter ที่กำหนดไว้ข้างต้น

In [ ]:
%%time
# สร้างฐานข้อมูลเวกเตอร์ (Vector Store) จากข้อความที่แบ่งไว้แล้ว (texts) โดยใช้ Chroma ซึ่งเป็นระบบฐานข้อมูลเวกเตอร์ความเร็วสูง และบันทึกลงโฟลเดอร์
db = Chroma.from_documents(texts, embeddings, persist_directory="db")
# สรุป ก็คือ สร้างฐานข้อมูลเวกเตอร์จากข้อความที่ได้ โดยใช้ ChromaDB และบันทึกไว้ในโฟลเดอร์ db เพื่อให้สามารถนำไปใช้ค้นหาข้อมูลแบบ semantic search ได้ในภายหลัง

## Llama 2 7B

In [ ]:
# โหลด tokenizer ที่เข้ากันได้กับโมเดล meta-llama/Llama-2-7b-hf เพื่อเตรียมใช้งานในการประมวลผลข้อความ ก่อนป้อนเข้าโมเดล
model_name_or_path = "meta-llama/Llama-2-7b-hf"
model_basename = "model"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

In [ ]:
# โหลดโมเดลสำหรับใช้งาน NLP โดยใช้ Transformers จาก Hugging Face ซึ่งเหมาะสำหรับการใช้งานโมเดล LLaMA หรือโมเดลขนาดใหญ่ทั่วไปในงาน RAG, Chatbot หรือ Text Generation
model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    device_map="auto", # ให้ transformers จัดการเลือกอุปกรณ์ที่เหมาะสมให้โดยอัตโนมัติ เช่น:ถ้ามี GPU → โหลดบน GPU
    torch_dtype="auto", # ให้ PyTorch จัดการชนิดของข้อมูล (เช่น float16, float32) ตามความเหมาะสม เพื่อประสิทธิภาพสูงสุด
    trust_remote_code=True # อนุญาตให้โหลด custom code ที่มากับโมเดลนั้นจาก Hugging Face
)
# สรุป เป็นการกำหนดค่าเำื่อให้โหลดโมเดล LLaMA (หรืออื่น ๆ) โดยปรับให้ใช้งานได้อัตโนมัติกับ hardware ที่เรามี พร้อมกำหนดชนิดข้อมูลที่เหมาะสม และรองรับ custom code จาก Hugging Face

In [ ]:
DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <<SYS>>
{system_prompt}
<</SYS>>

{prompt} [/INST]
""".strip()

In [ ]:
# TextStreamer ช่วยให้เห็นผลลัพธ์จากโมเดล LLaMA แบบไหลลื่นทีละคำ ✨ เหมาะกับการใช้ใน chatbot, live demo, หรือ interactive app ที่ต้องการ “stream response”
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

In [ ]:
# สร้าง text generation pipeline โดยใช้โมเดล LLaMA ที่โหลดไว้แล้ว เพื่อให้สามารถ generate ข้อความได้สะดวกในคำสั่งเดียว 
text_pipeline = pipeline(
    "text-generation", # ประเภท task คือ การ generate ข้อความ
    model=model, # โมเดลที่โหลดไว้แล้ว
    tokenizer=tokenizer, # tokenizer ที่ตรงกับโมเดล
    max_new_tokens=1024, # จำกัดจำนวนคำที่ generate สูงสุด
    temperature=1e-5, #temperature=0 อาจทำให้เกิด Error: temperature must be strictly positive
    top_p=0.95, # ใช้ nucleus sampling เพื่อควบคุมความสุ่ม
    repetition_penalty=1.15, # ป้องกันการวนซ้ำของข้อความเดิม
    streamer=streamer, # แสดงผลแบบสตรีมข้อความ real-time
)

# สรุป คือ สร้าง pipeline สำหรับให้โมเดล LLaMA สร้างข้อความ โดยควบคุมความสุ่ม และแสดงผลแบบ streaming ใช้ในการถาม-ตอบ หรือ generate ข้อความได้ทันที

In [ ]:
# แปลง Hugging Face pipeline ให้อยู่ในรูปของ LLM ที่ LangChain ใช้งานได้ พร้อมตั้งค่า temperature เพื่อควบคุมระดับความหลากหลายของข้อความที่โมเดลจะ generate
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0.7})

In [ ]:
# เตรียม prompt template สำหรับใช้ถาม LLM โดยดึง context (เนื้อหา) และ question (คำถาม) มาเติมในช่องที่กำหนดไว้
SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."

template = generate_prompt(
    """
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)

In [ ]:
# สร้าง Prompt Template ซึ่งเป็นรูปแบบข้อความ (prompt) ที่จะส่งให้กับ LLM เพื่อให้มันตอบคำถามตามโครงสร้างที่เรากำหนดไว้
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

In [ ]:
# สร้าง RAG Pipeline (Retrieval-Augmented Generation) แบบง่าย โดยใช้ RetrievalQA จาก LangChain เพื่อให้ LLM ตอบคำถามจากเอกสาร PDF ที่เราดึงข้อมูลไว้แล้ว
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,  # ใช้ LLM ที่เตรียมไว้ เช่น LLaMA ที่เชื่อมกับ text generation pipeline
    chain_type="stuff",  # รวมเอกสารทั้งหมดเข้าไปใน prompt เดียว (เหมาะกับข้อมูลไม่ยาวมาก)
    retriever=db.as_retriever(search_kwargs={"k": 2}),  # ค้นหาเอกสารที่ใกล้เคียงกับคำถามมากที่สุด 2 ชิ้น
    return_source_documents=True,  # ให้คืนเอกสารต้นฉบับที่ใช้ในการหาคำตอบมาด้วย
    chain_type_kwargs={"prompt": prompt},  # ใช้ prompt template ที่เรากำหนดเองไว้สำหรับการถาม-ตอบ
)

## Chat with Multiple PDFs

In [ ]:
result = qa_chain.invoke("What is the per share revenue for Meta during 2023?")

# .invoke() เป็นวิธีใหม่ (แทน .run() หรือ .call()) ตามมาตรฐานของ LangChain เวอร์ชันใหม่
# คำถามนี้จะถูกส่งไปที่ Retriever เพื่อค้นหาข้อมูลในเอกสาร PDF ที่ใกล้เคียง
# จากนั้นจะถูกส่งเข้า LLM (LLaMA) เพื่อสร้างคำตอบตาม prompt ที่เรากำหนดไว้

print(result['result'])  # แสดงคำตอบที่ได้จากโมเดล

In [ ]:
print(result['source_documents'])  # แสดงเอกสารที่ถูกใช้เพื่ออ้างอิงคำตอบ

In [ ]:
len(result["source_documents"])

In [ ]:
print(result["source_documents"][0].page_content)

In [ ]:
result =  qa_chain.invoke("What is the per share revenue for Tesla during 2023?")
result

In [ ]:
result =  qa_chain.invoke("What is the per share revenue for Nvidia during 2023?")
result

In [ ]:
print(result["source_documents"][1].page_content)

In [ ]:
result =  qa_chain.invoke("What is the estimated YOY revenue for Meta during 2023?")
result

In [ ]:
result =  qa_chain.invoke("What is the estimated YOY revenue for Tesla during 2023?")
result

In [ ]:
result =  qa_chain.invoke("What is the estimated YOY revenue for Nvidia during 2023?")
result

In [ ]:
result = qa_chain.invoke(
    "Which company is more profitable during 2023 Meta, Nvidia or Tesla and why?"
)
result

In [ ]:
result = qa_chain.invoke(
    "Choose one company to invest (Tesla, Nvidia or Meta) to maximize your profits for the long term (10+ years)?"
)
result

## References

- [Tesla Quarterly Report (Jul 21, 2023)](https://ir.tesla.com/_flysystem/s3/sec/000095017023033872/tsla-20230630-gen.pdf)
- [Meta Q2 2023 Earnings (Jul 26, 2023)](https://s21.q4cdn.com/399680738/files/doc_financials/2023/q2/Meta-06-30-2023-Exhibit-99-1-FINAL.pdf)
- [Nvidia Fiscal Q1 2024](https://s201.q4cdn.com/141608511/files/doc_financials/2024/q1/ecefb2b2-efcb-45f3-b72b-212d90fcd873.pdf)